# Headshot Cropper

Batch-process images: first use openCV to detect faces, then expand the face bounding box to max possible, then crop a circle out of the bounding box, finally soften the edges.

# Features

**1. 1-line code for face recognition and cropping into a circle image with feathered (adjustable) edges, on a transparent background**

```python
    result=blurEdge(cropImage(img, faceDetection(img_cv)), blur_radius, offset=0)
```

**2. Process multiple files and batch export to destination folder**

Input files are stored in `input_PNG_images` folder. The script reads every `*.png` file in that folder, and process them, and export the result for each image to the `result` folder. A zip file of the `result` folder is also recreated for convenience of downloading from web-hosted jupyter notebooks.

**Note:**

**1. If multiple faces are detected in a single photo, only the first face box will be extracted. The program does a generally good job at handling professional headshot, but it can also handle images with a certain level of complex backgrounds (depending on the accuracy of the openCV face detection)**

**2. For the simplicity of demo,  script only reads PNG images, but you can change this parameter in the `os.path.join(img_path, '*.png')` line**




## Examples

**Note: All resulting images resized to 200x200px. This parameter is user-adjustable. The extent of edge feather effect is also adjustable**

### **1. Simple background**

**Before**

![simple-bg](input_PNG_images/simple-background.png)

**After**

![simple-bg-after](result/simple-background_2.png)


### 2. Complex background

**Before**

![complex-1](input_PNG_images/complex_background1.png)


**After**

![complex-1-after](result/complex_background1_2.png)

**Before**

![complex-2](input_PNG_images/complex_background2.png)

**After**

![complex-2-after](result/complex_background2_2.png)

*Example pictures credit: freepik (obtained under CAS's image license with freepik)*

# Code

In [2]:
import cv2 as cv
from PIL import Image
from PIL import ImageDraw, ImageFilter
import glob
import os
import shutil

In [3]:
def faceDetection(img_cv):

    #load openCV Classifier
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

    #convert CV image to greyscale
    gray = cv.cvtColor(img_cv, cv.COLOR_BGR2GRAY)

    #detect faces
    faces = face_cascade.detectMultiScale(gray)

    #get bounding box for the face.
    face_lst=faces[0].tolist()   #Because this is a headshot, so there is only 1 face
    x1, y1, x2, y2=face_lst[0], face_lst[1], face_lst[0]+face_lst[2], face_lst[1]+face_lst[3]

    return (x1, y1, x2, y2)

In [4]:
def cropImage(img, face_box):  #use pillow RGB color mode images

    #get coordinates for the 2 corners of the face box
    x1, y1, x2, y2=face_box

    #create a draw object for the colored image
    draw=ImageDraw.Draw(img)

    #pass in the face bounding box
    #draw.rectangle((x1, y1, x2, y2))

    #draw outline for the bounding box
    #draw.rectangle((x1, y1, x2, y2), outline="white")
    #display(img)

    #get the width, height of the face box
    w, h=x2-x1, y2-y1

    #make the facebox a square so it's easier to cut a circle out of it later on
    if w>h:
        diff=(w-h)/2
        y1-=diff
        y2+=diff
    else:
        diff=(h-w)/2
        x1-=diff
        x2+=diff

    #now expand the face box to max possible. Start with alpha=0, gradually increase alpha
    alpha=0

    # safe zone to avoid cutting outside of the border
    safe = 0.2

    #make sure face box doesn't go beyond the image borders!
    while (x1>0 and x2 < img.size[0]-safe and y1>0 and y2<img.size[1]-safe):
        x1-=alpha
        y1-=alpha
        x2+=alpha
        y2+=alpha
        alpha+=0.02  #step size for alpha

    #visulize the expanded face box
    #draw.rectangle((x1, y1, x2, y2), outline="white")
    #display(img)

    #crop the image using the expanded face box
    newImg=img.crop((x1, y1, x2, y2))

    return newImg

#blur the edges
def blurEdge(img, blur_radius, offset=0):
    offset = blur_radius * 2 + offset
    mask = Image.new("L", img.size, 0) #0 is black
    draw = ImageDraw.Draw(mask)
    draw.ellipse((offset, offset, img.size[0] - offset, img.size[1] - offset), fill=255) #fill with white
    mask = mask.filter(ImageFilter.GaussianBlur(blur_radius))

    result = img.copy()
    result.putalpha(mask) #pixels=0(black) will disappear, pixels=255 (white) will show through

    return result

In [5]:
def makeDir(output_path):
    """
    Handling directory of result files. Results will be stored in a folder called `result`. The program checks if such folder exsits. If it exists, its current contents will be disgarded. If it does not exist, the folder will be greated.
    :return: None
    """
    try:
        shutil.rmtree(output_path)  #delete 'result' folder and all its current contents if it exist
    except:
        pass  #if it doesn't exist then don't do anything
    os.mkdir(output_path) #make a new `result` folder

In [8]:
if __name__=="__main__":

    #specify input file path and output file path
    img_path = 'input_PNG_images/'
    output_path='result/'

    makeDir(output_path)

    # go through every PNG file in the input_PNG_logo folder, process it, then save the output in the result folder
    for filename in glob.glob(os.path.join(img_path, '*.*')): # ONLY processs *.png file. Modify to suite your need
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            fn=(filename.split('/')[-1]).split('.')[0] #get the name of the image. E.g. "harbormed" instead of "input_PNG_Logo/harbormed.png"
            new_filename=output_path+fn+"_2.png"

            #read image into openCV
            img_cv = cv.imread(filename)

            #read image into PIl and convert to RGB
            img=Image.open(filename).convert("RGB")

            #set blur radius
            blur_radius=2

            #process the image
            result=blurEdge(cropImage(img, faceDetection(img_cv)), blur_radius, offset=0)

            #resize the image
            result.resize((800, 800)).save(new_filename)  #export as 800 * 800


    shutil.make_archive("result", 'zip', "result") #zip up file for easy download in web-based Jupyter
